# Scraping A Website

In [1]:
url = "https://www.ft.com/content/4a24a1d2-f756-4776-9c08-3a671e063e6a#post-247feacc-d098-49ef-8a0b-bf51b4723afc"


In [13]:
import json
with open("../.ft-headers.json", "r") as f:
    headers = json.load(f)

In [5]:
import httpx

with httpx.Client(headers=headers) as client:
    response = client.get(url)

print(response.status_code)
# print(response.text)  # or response.content if you prefer the raw bytes


200


In [6]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "lxml")

In [7]:
print("\n".join([p.text.strip() for p in soup.findAll('p')]))

Today’s top headlines:
Federal Reserve cuts rates by half a point and signals era of easing has begun
UK inflation holds steady in August
Boeing to temporarily furlough workers amid strike
More Hizbollah devices explode across Lebanon
Indonesia makes quarter-point interest rate cut in surprise move
Edited by Alexandra White, Zehra Munir, Jaren Kerr, Oliver Ralph, Jonathan Wheatley, Maxine Kelly and George Russell
Colby Smith in Washington
The Federal Reserve cut its benchmark interest rate by half a percentage point on Wednesday and signalled more reductions would follow, launching its first easing cycle since the onset of the pandemic.
The US central bank’s first cut in more than four years leaves the federal funds rate at a range of 4.75 per cent to 5 per cent. Michelle Bowman, a governor on the Federal Open Market Committee, voted against the decision, favouring a quarter-point reduction.
The half-point cut is larger than the Fed’s more customary quarter-point pace and suggests the 

## Scraping FT "Blog Posts"

In [24]:
url = "https://www.ft.com/content/4a24a1d2-f756-4776-9c08-3a671e063e6a#post-247feacc-d098-49ef-8a0b-bf51b4723afc"
splits = url.split('#')
if (len(splits) > 1):
    post_id = splits[-1]
post_id

'post-247feacc-d098-49ef-8a0b-bf51b4723afc'

In [35]:
post_id_element = soup.find(id=post_id)
title = post_id_element.find('h2').get_text()
content = "\n".join([p.text.strip() for p in post_id_element.find_all('p')])
time = post_id_element.find('time')
print(title)
print(content)
print(time.get('datetime'))

UK inflation holds steady in August
Sam Fleming in London
UK inflation held steady at 2.2 per cent in August, leaving the door open for the Bank of England to cut interest rates further this year.
The annual increase in consumer prices, reported by the Office for National Statistics on Wednesday, was unchanged from July’s rate and matched the expectations of economists polled by Reuters.
Inflation has now been above the BoE’s 2 per cent target for two consecutive months.
The BoE cut rates by a quarter point to 5 per cent in August — the first reduction in more than four years — and signalled it would lower borrowing costs again if inflation eased further.
The bank’s Monetary Policy Committee will announce its latest rate decision on Thursday.
2024-09-18T06:03:07.631Z


# Getting Article List

In [7]:
import httpx
url = "https://www.ft.com/search"

params = {
    "q": "uk inflation",
    "page": 1,
    "sort": "relevance", # or "date"
    "isFirstView": "false" # i assume true means don't show articles that have been seen before
}

with httpx.Client() as client:
    response = client.get(url, params=params)

print(response.status_code)

200


In [10]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(response.text, "lxml")
div_items = soup.find_all('div', class_='search-item')

for item in div_items:
    a_tag = item.find('a', class_='js-teaser-heading-link')
    print(a_tag['href'])
    print(a_tag.text)


https://www.ft.com/content/85d0bd6f-b7ce-4d69-b3af-9bc336b73fc8
UK inflation holds steady at 2.2% in August
/content/2095faeb-c613-4fe2-ac39-7a2f3d4e3027#post-d2da4d94-0e99-4601-9a02-e1f96adb7ac6
UK inflation is below 2% target
/content/2bd1b3fa-0bb4-4844-87e6-2e292b3cdb41#post-e09cbd49-90ce-4fcb-909c-a69eabc4ee63
UK inflation falls to 1.7% in September
https://www.ft.com/content/df2191ce-9a34-4a7b-901c-3513db25d585
UK inflation fading faster than expected, says BoE governor
https://www.ft.com/content/18df43a2-ac5d-47e0-b315-651a2233a631
Pound falls as UK inflation declines more than expected to 1.7%
/content/4a24a1d2-f756-4776-9c08-3a671e063e6a#post-247feacc-d098-49ef-8a0b-bf51b4723afc
UK inflation holds steady in August
https://www.ft.com/content/a9ec45a8-6f60-4955-80c8-c1d5fb6105fb
UK set for slower inflation than Eurozone and US
https://www.ft.com/content/4b9acf23-bec8-4056-9850-da038a8d3b28
UK inflation could stay near 2% target for three years, says BoE rate-setter 
https://www.f